## Adding sub-annual time slices and storage
This script adds seasonality and storage to Maldives model/scenario
For more information on seasonality, please refer to [this tutorial](<https://github.com/iiasa/message_ix/blob/master/tutorial/westeros/westeros_seasonality.ipynb>)

In [1]:
# Importing required packages
import itertools
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import ixmp
import message_ix

from message_ix.utils import make_df

In [2]:
mp = ixmp.Platform(name='local')

In [3]:
# Loading a baseline scenario
model = "Maldives energy model"
#scen = "baseline"
#version = 1
base = message_ix.Scenario(mp, model=model, scenario='baseline')

# Cloning to a new scenario for editing
scen = base.clone(model, 'time-slice', 
                  keep_solution=False)

# Checking out the scenario for editing
scen.check_out()

### 1. Representing sub-annual time slices
#### 1.1. Modifying sets

In [4]:
# Adding sub-annual time steps
time_steps = ['day', 'night']
scen.add_set('time', time_steps)

In [5]:
# Defining a new temporal level
time_level = 'daylight'
scen.add_set('lvl_temporal', time_level)

# Adding temporal hierarchy
for t in time_steps:
    scen.add_set('map_temporal_hierarchy', [time_level, t, 'year'])

#### 1.2 Modifying parameters

In [6]:
# Adding duration time
for t in time_steps:
    scen.add_par('duration_time', [t], 1/len(time_steps), '-')

We define a function for manupulating the data of parameters for each sub-annual time slice. This is used for changing demand, input/output, var_cost, etc.

In [7]:
# A function for adding sub-annual data to a parameter
def yearly_to_season(scen, parameter, data, filters=None):
    if filters:
        old = scen.par(parameter, filters)
    else:
        old = scen.par(parameter)
    scen.remove_par(parameter, old)
    
    # Finding "time" related indexes and updating the data
    time_idx = [x for x in scen.idx_names(parameter) if 'time' in x]
    for h in data.keys():
        new = old.copy()
        for time in time_idx:
            new[time] = h
        new['value'] = data[h] * old['value']
        scen.add_par(parameter, new)

In [8]:
# Modifying demand for each time slice
# Electricity demand
demand_data = {'day': 0.4, 'night': 0.60}
filters = {'commodity': 'electricity'}
yearly_to_season(scen, 'demand', demand_data) #added =None

# Here, I just divid them the yearly demand for day and night equally
demand_data = {'day': 0.50, 'night': 0.50}
filters = {'commodity': ['cooling', 'freshwater']}
yearly_to_season(scen, 'demand', demand_data) #added =None

In [9]:
# Modifying input and output parameters for each time slice
# Output of solar PV at night zero
pv_data = {'day': 1, 'night': 0}
pv_filters = {'technology': 'solar_pv_ppl'}
yearly_to_season(scen, 'output', pv_data, pv_filters)

# Output of other technologies
fixed_data = {'day': 1, 'night': 1}
other_filters = {'technology': ['oil_ppl', 'battery_ppl', 'ac', 'ro_pl', 'swac', 'dscd']}
yearly_to_season(scen, 'output', fixed_data, other_filters)

# Input of all technologies
yearly_to_season(scen, 'input', fixed_data)

In [10]:
# Modifying capacity factor
# Yearly capacity factor of solar_pv in the baseline scenario
cf_solar = scen.par('capacity_factor', {'technology': 'solar_pv_ppl'})['value'].mean()

# Converting yearly capacity factor to day-night (multiplying CF by 2 for day)
cf_data = {'day': 2, 'night': 0} 
yearly_to_season(scen, 'capacity_factor', cf_data, pv_filters)

# Capacity factor of other technologies remains unchanged in each time slice
yearly_to_season(scen, 'capacity_factor', fixed_data, other_filters)


In [11]:
# Modifying initial penetration rates for each time slice
half_data = {'day': 0.5, 'night': 0.5} 
yearly_to_season(scen, 'initial_activity_up', half_data)

In [12]:
# Modifying historical activity 
hist_data = {'day': 1, 'night': 0}
yearly_to_season(scen, 'historical_activity', hist_data, pv_filters)
# For others divide equally between day and night
hist_data = {'day': 0.5, 'night': 0.5}
yearly_to_season(scen, 'historical_activity', hist_data, other_filters)

In [13]:
# Modifying variable cost
yearly_to_season(scen, 'var_cost', fixed_data)

In [14]:
# Removing output of solar_pv at night

In [15]:
scen.commit(comment='introducing seasonality')
scen.set_as_default()

In [16]:
scen.solve()

### 2. Representing storage

In [17]:
# Cloning to a new scenario for adding storage
sc = scen.clone(model, 'storage', 'storage_solution', keep_solution=False)

# Checking out the scenario for editing
sc.check_out()

#### 2.1 Configuring storage sets

In [18]:
# Adding level of storage
sc.add_set('level', 'store')

# Adding storage technologies (based on the notation of: reservoir, charger, and discharger)
sc.add_set('technology', ['battery_stor', 'dummay', 'battery_ppl'])

# Adding a storage commodity
sc.add_set('commodity', ['dummay'])

# Specifying storage reservoir technology
sc.add_set('storage_tec', 'battery_stor')

# Specifying storage level
sc.add_set('level_storage', 'store')

# Adding mapping for storage and charger/discharger technologies
for tec in ['dummay', 'battery_ppl']:
    sc.add_set('map_tec_storage', ['Maldives', tec, 'battery_stor', 'store',
                                     'electricity'])

#### 2.2. Configuring storage parameters

In [19]:
# Adding the order of time slices
sc.add_par('time_order', ['daylight', 'day'], 1, '-')
sc.add_par('time_order', ['daylight', 'night'], 2, '-')

In [20]:
# Adding output and input for storage technologies
# Loading the output of battery
data = sc.par('output', {'technology': 'battery_ppl'})

# Order of data: ['commodity', 'level']
output_spec = {'battery_stor': ['dummay', 'final'],
               'dummay': ['electricity', 'store'],
               }
for tec, item in output_spec.items():
    data['technology'] = tec
    data['commodity'] = item[0]
    data['level'] = item[1]
    sc.add_par('output', data)

# Configuring input parameter
data = data.rename({'node_dest': 'node_origin', 'time_dest': 'time_origin'}, axis=1)
input_spec = {'battery_stor': ['dummay', 'final'],
              'dummay': ['electricity', 'final'],
              'battery_ppl': ['electricity', 'store'],
              }

for tec, item in input_spec.items():
    data['technology'] = tec
    data['commodity'] = item[0]
    data['level'] = item[1]
    sc.add_par('input', data)


In [21]:
# Adding storage self-discharge (as %) and initial content (assuming 3% loss of battery charge between day and night (6%/day))
for year, h in itertools.product(set(sc.set('year')), time_steps):
    storage_spec = ['Maldives', 'battery_stor', 'store', 'dummay', year, h]
    sc.add_par('storage_self_discharge', storage_spec, 0.03, '%')

    # Adding initial content of storage (optional)
    initial_value = 0
    storage_spec = ['Maldives', 'battery_stor', 'store', 'dummay', year, 'day']
    sc.add_par('storage_initial', storage_spec, initial_value, 'MWa')

#### Committing and solving

In [22]:
sc.commit('storage setup added')
sc.solve(var_list=['STORAGE', 'STORAGE_CHARGE', 'STORAGE_INIT'])

at.ac.iiasa.ixmp.exceptions.IxException: at.ac.iiasa.ixmp.exceptions.IxException: There exists no variable 'STORAGE'!

In [42]:
# The battery doesn't work as it is expensive with PV (oil_ppl supplies all demand)
sc.var('STORAGE')

RuntimeError: unhandled Java exception: There exists no variable 'STORAGE'!

In [43]:
# A new scenario with lower cost of battery or bound on oil
sc_lo = sc.clone(scenario='storage-oil-bound', keep_solution=False)
sc_lo.check_out()

In [44]:
# Reducing inv_cost
for parname in ['inv_cost']:
    df = sc_lo.par(parname, {'technology': 'battery_ppl', })
    df['value'] *= 1
    sc_lo.add_par(parname, df)

In [45]:
# bound on activity of oil at night
base_act = {
    'node_loc': 'Maldives',
    'year_act': list(set(sc_lo.set('year'))),
    'mode': 'standard',
    'time': 'night',
    'unit': 'MWa',
}

new_activity = {'oil_ppl': 100}

for tec, val in new_activity.items():
    df = make_df(base_act, technology=tec, value=val)
    sc_lo.add_par('bound_activity_up', df)

In [46]:
sc_lo.commit('')
sc_lo.solve(var_list=['STORAGE', 'STORAGE_CHARGE', 'STORAGE_INIT'])

at.ac.iiasa.ixmp.exceptions.IxException: at.ac.iiasa.ixmp.exceptions.IxException: There exists no variable 'STORAGE'!

In [168]:
sc_lo.var('STORAGE')

,node,technology,level,commodity,year,time,lvl,mrg
0,Maldives,battery_stor,store,electricity,2025,year,0.000000,0.000000e+00
1,Maldives,battery_stor,store,electricity,2025,day,186.004677,0.000000e+00
2,Maldives,battery_stor,store,electricity,2025,night,0.000000,4.940656e-324
3,Maldives,battery_stor,store,electricity,2030,year,0.000000,0.000000e+00
4,Maldives,battery_stor,store,electricity,2030,day,211.740177,0.000000e+00
5,Maldives,battery_stor,store,electricity,2030,night,0.000000,4.940656e-324
6,Maldives,battery_stor,store,electricity,2035,year,0.000000,0.000000e+00
7,Maldives,battery_stor,store,electricity,2035,day,224.308677,0.000000e+00
8,Maldives,battery_stor,store,electricity,2035,night,0.000000,4.940656e-324
9,Maldives,battery_stor,store,electricity,2040,year,0.000000,0.000000e+00


In [1]:
mp.close_db()

NameError: name 'mp' is not defined